In [271]:
import numpy as np

r_lambda = 40
nf = 200
alpha = 40

# original rating matrix
R = np.array([[2, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0],
              [3, 3, 4, 0, 3, 0, 0, 2, 2, 0, 0],
              [5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 7, 0, 0, 5, 0],
              [4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5],
              [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4],
              [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
              [0, 0, 0, 3, 0, 0, 0, 0, 4, 5, 0]])
print(R.shape)

(10, 11)


In [272]:
# nf : dimension of latent factors
# nu : number of users
# ni : number of items

nu = R.shape[0]
ni = R.shape[1]

# X, Y : latent matrix of users and items
X = np.random.rand(nu, nf) * 0.01
Y = np.random.rand(ni, nf) * 0.01

print(X)
print(Y)

[[0.00937501 0.00894192 0.00111307 ... 0.00130772 0.00115214 0.00720873]
 [0.00146159 0.00896956 0.00942212 ... 0.00407354 0.00108615 0.0075778 ]
 [0.00901456 0.0038649  0.00020889 ... 0.00718687 0.00585541 0.0060983 ]
 ...
 [0.00685747 0.00700845 0.00460261 ... 0.00496061 0.00917623 0.00049663]
 [0.00191428 0.00637595 0.00815958 ... 0.0057553  0.0035395  0.00644017]
 [0.00663356 0.00381629 0.00871886 ... 0.00098702 0.00708245 0.00388322]]
[[0.00897268 0.00284219 0.00820363 ... 0.00147741 0.00999528 0.00638416]
 [0.00065271 0.00873268 0.00225456 ... 0.00634664 0.00408266 0.00450254]
 [0.00505061 0.00334195 0.00893984 ... 0.00846426 0.00531104 0.00151123]
 ...
 [0.00366328 0.00984407 0.00095266 ... 0.00120807 0.00974001 0.00895036]
 [0.00789468 0.00019431 0.00742314 ... 0.00029259 0.00846265 0.00944247]
 [0.00962856 0.00511733 0.0015536  ... 0.00766279 0.00455305 0.00234931]]


In [273]:
# Pui = 1 if Rui > 0
# Pui = 0 if Rui = 0
P = np.copy(R)
P[P > 0] = 1
print(P)

[[1 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 1 0]
 [1 1 1 0 1 0 0 1 1 0 0]
 [1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 0]
 [1 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 1 0 0 0 0 1 1 0]]


In [274]:
# Cui = 1 + alpha * Rui
C = 1 + alpha * R
print(C)

[[ 81   1   1 161 161   1   1   1   1   1   1]
 [  1   1   1   1   1   1   1   1   1   1  41]
 [  1   1   1   1   1   1   1  41   1 161   1]
 [121 121 161   1 121   1   1  81  81   1   1]
 [201 201 201   1   1   1   1   1   1   1   1]
 [  1   1   1   1   1   1 281   1   1 201   1]
 [161   1 161   1   1   1   1   1   1   1 201]
 [  1   1   1   1   1 161   1   1   1   1 161]
 [  1   1   1   1   1   1 201   1   1 201   1]
 [  1   1   1 121   1   1   1   1 161 201   1]]


In [277]:
# C : confidence matrix
# P : predict matrix
# X : user latent matrix
# Y : item latent matrix
# r_lambda : regularization lambda
def confidence_loss(C, P, xTy, X, Y, r_lambda):
    predict_error = np.square(P - xTy)
    print('predict loss:', np.sum(predict_error))
    confidence_error = np.sum(C * predict_error)
    print('confidence loss:', confidence_error)
    regularization = r_lambda * (np.sum(np.square(X)) + np.sum(np.square(Y)))
    print('regularization: ', regularization)
    return confidence_error + regularization

In [278]:
def optimize_user(X, Y, C, P, nu, nf, r_lambda):
    yT = np.transpose(Y)
    for u in range(nu):
        Cu = np.diag(C[u])
        yT_Cu_y = np.matmul(np.matmul(yT, Cu), Y)
        lI = np.dot(r_lambda, np.identity(nf))
        yT_Cu_pu = np.matmul(np.matmul(yT, Cu), P[u])
        X[u] = np.linalg.solve(yT_Cu_y + lI, yT_Cu_pu)

def optimize_item(X, Y, C, P, ni, nf, r_lambda):
    xT = np.transpose(X)
    for i in range(ni):
        Ci = np.diag(C[:, i])
        xT_Ci_x = np.matmul(np.matmul(xT, Ci), X)
        lI = np.dot(r_lambda, np.identity(nf))
        xT_Ci_pi = np.matmul(np.matmul(xT, Ci), P[:, i])
        Y[i] = np.linalg.solve(xT_Ci_x + lI, xT_Ci_pi)
        
        
predict = np.matmul(X, np.transpose(Y))
initial_loss = confidence_loss(C, P, predict, X, Y, r_lambda)
print('initial loss: ', initial_loss)
print('---------------------------------------')

for i in range(30):
    optimize_user(X, Y, C, P, nu, nf, r_lambda)
    optimize_item(X, Y, C, P, ni, nf, r_lambda)
    predict = np.matmul(X, np.transpose(Y))
    loss = confidence_loss(C, P, predict, X, Y, r_lambda)
    print('loss: ', loss)
    print('---------------------------------------')
predict = np.matmul(X, np.transpose(Y))
print([predict])

predict loss: 23.25926034929978
confidence loss: 90.49493283569099
regularization:  705.3022907734701
initial loss:  795.797223609161
---------------------------------------
predict loss: 23.27289375471135
confidence loss: 90.51327734180576
regularization:  705.2833161340757
loss:  795.7965934758814
---------------------------------------
predict loss: 23.286054678402046
confidence loss: 90.53095335312098
regularization:  705.2650875611698
loss:  795.7960409142908
---------------------------------------
predict loss: 23.298692447847934
confidence loss: 90.54790273860365
regularization:  705.247652322921
loss:  795.7955550615246
---------------------------------------
predict loss: 23.31077926942078
confidence loss: 90.56409563494508
regularization:  705.2310312614192
loss:  795.7951268963642
---------------------------------------
predict loss: 23.32230393653068
confidence loss: 90.57952257054737
regularization:  705.2152262717334
loss:  795.7947488422808
------------------------------